In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
import torchvision
import numpy as np
from nilearn import plotting
import clinicadl
import pandas as pd
import matplotlib.pyplot as plt
from torchinfo import summary
from sklearn.metrics import log_loss
from collections import OrderedDict
from PIL import Image
from tqdm import tqdm
from math import floor
import random
import time
import os
import json
import re

# torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# clinicaDL
from clinicadl.tools.tsv.data_split import create_split
from clinicadl.tools.deep_learning.data import generate_sampler, return_dataset, MRIDataset, get_transforms
from torch.utils.data import DataLoader
from clinicadl.tools.deep_learning.cnn_utils import train, get_criterion, test
from clinicadl.tools.deep_learning.models.random import RandomArchitecture
from clinicadl.tools.deep_learning import EarlyStopping

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

# visualization
from scipy.ndimage import zoom
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt

%matplotlib inline

/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [3]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
from tools.explanations.evaluation import *
from tools.explanations.GradCam import *
from tools.explanations.guided_backprop import *
from train.train_CNN import *
from tools.callbacks import *
from tools.data import *
from tools.explanations.visualization import *
from tools.models.CN5_FC3_3D import *
from tools.settings import *

In [5]:
data_path = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021.tsv'
summary_path = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021_summary.tsv'
# df_data = pd.read_csv(data_path,sep='\t',nrows=10)
# df_summary = pd.read_csv(summary_path,sep='\t',nrows=10)

In [6]:
pipeline_name='t1-volume'
atlas_id='AAL2'

# Train Single CNN

## Initialization

In [1]:
# global parameters
caps_directory = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/'
batch_size = 16
num_workers = os.cpu_count()
learning_rate = 1e-4
weight_decay = 1e-4

NameError: name 'os' is not defined

In [ ]:
# load dataframes
AD = pd.read_csv('subjects/AD.tsv',sep='\t')
CN = pd.read_csv('subjects/CN.tsv',sep='\t')

# remove samples with NaN
AD.drop(AD[AD.isna().sum(axis=1) > 0].index, inplace=True)
CN.drop(CN[CN.isna().sum(axis=1) > 0].index, inplace=True)

# split data between training and validation sets
training_df, valid_df = create_split('AD', AD, 'diagnosis',0.2)
df_CN = create_split('CN', CN, 'diagnosis',0.2)
training_df = training_df.append(df_CN[0]).reset_index().iloc[np.array([0,1,2,-1,-2,-3])]
valid_df = valid_df.append(df_CN[1]).reset_index().iloc[np.array([0,1,2,-1,-2,-3])]

# drop index column
training_df.drop(columns = ['index'], inplace=True)
valid_df.drop(columns = ['index'], inplace=True)

train_transforms, all_transforms = get_transforms('image', minmaxnormalization=True, data_augmentation=None )
# fetch volumetric data
stds, df_add_data = fetch_add_data(training_df)

# all_transforms = torchvision.transforms.Compose([])

In [ ]:
data_train = MRIDatasetImage(caps_directory, training_df, df_add_data=df_add_data,all_transformations=all_transforms) #train_transformations=all_transforms
data_valid = MRIDatasetImage(caps_directory, valid_df, df_add_data=df_add_data, all_transformations=all_transforms) #train_transformations=all_transforms,


# sampler
train_sampler = generate_sampler(data_train)
valid_sampler = generate_sampler(data_valid)
# loaders
train_loader = DataLoader(data_train,
                         batch_size=batch_size,
                         sampler=train_sampler,
                         num_workers=num_workers,
                         pin_memory=True)

valid_loader = DataLoader(data_valid,
                         batch_size=batch_size,
                         sampler=valid_sampler,
                         num_workers=num_workers,
                         pin_memory=True)

## Training

In [ ]:
# get sample
sample = data_train[0]
# build model
model = Net(sample, [8, 16, 32, 64, 128])
# if torch.cuda.is_available():
#     print("To cuda")
#     model.cuda()
model.summary(batch_size=16)

In [ ]:
# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# record losses
train_losses = dict()
test_losses = dict()

# callbacks
ES = EarlyStopping(patience=5)
MC = ModelCheckpoint()

print("Beginning of the training")

# training
for epoch in range(2):
    update_dict(train_losses, train(epoch, model, optimizer, train_loader, to_cuda=True))
    update_dict(test_losses, test(model, valid_loader, to_cuda=True, rescaling=stds))

In [ ]:
for k in range(1,5):
    print(np.linalg.norm(getattr(getattr(model,'branch' + str(k)), 'b' + str(k) + '-conv').weight.grad.data.cpu()))

## Loss visualization

In [ ]:
def plot_losses(dict_losses, title=""):
    """
    Plot the different losses.
    
    Args:
        dict_losses: dictionnary of losses
    """
    plt.figure()
    plt.title(title)
    for key in dict_losses.keys():
        plt.plot(dict_losses[key], label=key)
    plt.legend()
    plt.show()
    
def plot_tensor(X):
    x = np.transpose(X[0], (1,2,0))
    x = (x-x.min())/x.max()
    plt.imshow(x)

In [ ]:
metric_path = 'results/models/model_5/'

train_metrics = pd.read_csv(metric_path + 'train_losses.csv')
test_metrics = pd.read_csv(metric_path + 'val_losses.csv')

In [ ]:
plot_losses(train_metrics[['disease']], "Training losses")

In [ ]:
plot_losses(test_metrics[['disease']], "Val losses")

## Explanations

In [ ]:
args = dict()
args['dataset'] = 'val'
args['preprocessing'] = 't1-linear'
args['convolutions'] = [8,16,32,64,128]
args['dropout'] = 0.2
args['model_path'] = 'results/models/model_85'


caps_directory = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/'
path = 'results/models/model_85/'
training_df = pd.read_csv(os.path.join(args['model_path'], 'training_df.csv'))
valid_df = pd.read_csv(os.path.join(args['model_path'], 'valid_df.csv'))

# get transformations
train_transforms, all_transforms = get_transforms('image', minmaxnormalization=True, data_augmentation=None)

# fetch volumetric data (useless in practice)
stds, df_add_data = fetch_add_data(training_df)

# build data loader
if args['dataset'] == 'train':
    data_loader = MRIDatasetImage(caps_directory, training_df, df_add_data=df_add_data,
                                  preprocessing=args['preprocessing'], all_transformations=all_transforms)
elif args['dataset'] == 'val':
    data_loader = MRIDatasetImage(caps_directory, valid_df, df_add_data=df_add_data, preprocessing=args['preprocessing'],
                                  all_transformations=all_transforms)
else:
    raise Exception('No dataset.')

# get sample
sample = data_loader[0]
# build model
model = Net(sample, args['convolutions'], args['dropout'], False).cuda()
# load pretrained weights on validation set
saved_data = torch.load(os.path.join(args['model_path'], 'test_best_model.pt'))
model.load_state_dict(saved_data['model_state_dict'])

# initialize interpretability method
GC = GradCam(model)

### old

In [ ]:
sample = data_train[0]
img = sample['image'].float()
img.shape

In [ ]:
# load model
saved_data = torch.load(os.path.join(path, 'test_best_model.pt'))

model.load_state_dict(saved_data['model_state_dict'])

# select one image
img = sample['image'].float()

mode = 'cuda'
if mode == 'cuda':
    model = model.cuda()
    img = img.cuda()
else:
    model = model.cpu()
    img = img.cpu()

In [ ]:
ar = np.load('results/models/model_19/attribution_maps/GC/val/age/sub-ADNI002S1018_ses-M00.npy')

### Loss visualization

In [ ]:
saved_data = torch.load('results/models/model_19/last_model.pt')

In [ ]:
saved_data.keys()

In [ ]:
train_metrics = saved_data['val_metrics']

In [ ]:
train_metrics['b3_MeanSquaredError']

In [ ]:
pd.DataFrame(train_metrics)['b3_MeanSquaredError'].plot()

### Guided Backpropagation

In [ ]:
GBP = GuidedBackprop(model)
attention_maps = GBP.generate_gradients(img)

In [ ]:
visualize_explanations(img.cpu(), attention_maps)

### GradCAM

In [ ]:
a0 = time.time()
GC = GradCam(model)
img = sample['image']
attentions = GC.get_explanations(img.cuda(), resize=False, to_cpu=True)
print(time.time() - a0)

In [ ]:
a0 = time.time()
model.cuda()
GC = GradCam(model)
img = data_loader[1]['image']
attentions = GC.generate_cam(img.cuda(), branch='branch1', resize=True, to_cpu=True)
print(time.time() - a0)

In [ ]:
visualize_explanations(img, {'branch1':attentions}, targets='disease')

In [ ]:
# for key in attentions:
#     attentions[key] = attentions[key].cpu()
visualize_explanations(img, attentions) #, targets='volumes')

In [ ]:
a0 = time.time()
GC = GradCam(model.cuda(), target_layer='conv2')
img = sample['image'].float()
attentions = GC.get_explanations(img.cuda(), resize=True, to_cpu=True)
print(time.time() - a0)

In [ ]:
visualize_explanations(img, attentions)

In [ ]:
visualize_explanations(img, attentions, targets='disease')

### max-sensitivity

In [ ]:
a,b,c,d = [], [], [], []
n_max = 500
step = 20
for k in tqdm(range(1, n_max, step)):
    np.random.seed(0)
    random.seed(0)
    torch.manual_seed(0)
    resu = max_sensitivity(img, GC, k)
    a.append(resu['branch1'])
    b.append(resu['branch2'])
    c.append(resu['branch3'])
    d.append(resu['branch4'])

In [ ]:
t = time.time()
max_sensitivity(img, GC, N=1)
print(time.time() - t)

In [ ]:
for elem in (a, b, c ,d):
    plt.figure()
    plt.plot(np.arange(1,n_max,step), elem)

### MoRF

In [ ]:
attention = attentions['branch1']

In [ ]:
ids = np.unravel_index(np.argsort(-attention, axis=None), attention.shape)

In [ ]:
batch_size = 16
group_size = 20000
batch_X = torch.tile(img[None,...], (batch_size,1,1,1,1))
for k in range(1, batch_size):
    index = k*group_size
    batch_X[k,0,ids[0][:index], ids[1][:index], ids[2][:index]] = 0

In [ ]:
model2 = model.cpu()

In [ ]:
resu = model2(batch_X)

In [ ]:
img = sample['image'].float()
np.prod(img.shape)//8

In [ ]:
def MoRF(X, model, exp_method, K=None, group_size=20000, AUC=False, batch_size=16, to_cuda=False):
    """
    Most relevant first: measures the reliability of an explanation by testing 
    how fast the output decreases, while we progressively remove information (e.g., perturb pixels) 
    from the input 𝑥𝑥 (e.g., image), that appears as the most relevant by the explanation.
    Args:
        X: tensor, brain image, with shape (1, n_channels, depth, height, width). The two first dimensions
            are optional.
        exp_method: explanation method. Must have a get_explanations(self, input_image) attribute function
            which takes an image as input and returns a dictionary mapping branches to explanation maps
        K: number of group of relevant pixels to remove
        group_size: int, size of a group of pixels to remove
        AUC: bool. If True: compute and return area under the curve obtained after removing successively 
            the K most relevant pixels.
        batch_size: int, number of images passed to the model each time
            
    TO DO:
        - add several methods to perturb pixels
    """
    if to_cuda and torch.cuda.is_available():
        X = X.cuda()
        
    if K is None:
        K = np.prod(X.shape)//8
    
    # reshpae X if necessary
    while len(X.shape) < 5:
        X = X[None,...]
    
    # original predictions
    preds = model(X)
    # explanations for original image
    expls = exp_method.get_explanations(X, resize=True)
    # explanations for new images
    new_preds = dict()
    
    # def update_dict
    
    for target in expls:
        # Indices of the sorted elements of the explanations:
        ids = np.unravel_index(np.argsort(-expls[target], axis=None), expls[target].shape)
    
        if AUC:
            # number of 
            removed_pixels = 0
            while removed_pixels < K:
                # create batch of images
                bs = min(batch_size,(K - removed_pixels)%group_size)
                batch_X = torch.tile(X, (bs,1,1,1,1))
                for k in range(1, batch_size):
                    index = k*group_size
                    batch_X[k,0,ids[0][:index], ids[1][:index], ids[2][:index]] = 0
                    new_preds[target] = model(batch_X)
        else:
            # compute MoRF removing the K most relevant pixels
            batch_X = X.copy()
            batch_X[0,0,ids[0][:K], ids[1][:K], ids[2][:K]]
            new_preds[target] = model(batch_X)
        

### Explanations analysis

In [35]:
from tools.explanations.analysis.utils import *
from tools.settings import *
import nibabel as nib
from tools.data import *

In [37]:
atlas_tsv, atlas_map = load_atlas()
atlas_tsv = pd.read_csv('atlas/atlas-AAL2_space-MNI152NLin2009cSym_dseg.csv')
#atlas_tsv.roi_value = atlas_tsv.roi_value.apply(ast.literal_eval)

In [ ]:
att_map = np.load(os.path.join('results/models/model_85/attribution_maps/GC/val/age', 'sub-ADNI018S0425_ses-M00.npy'))
#resized_att_map = zoom(att_map, atlas_map.shape/ np.array(att_map.shape))

In [ ]:
# https://stackoverflow.com/questions/9786102/how-do-i-parallelize-a-simple-python-loop

In [ ]:
def compute_scores(data_path, atlas_tsv, atlas_map):
    scores = {}
    # fetch file names
    files = [file for file in os.listdir(data_path) if os.path.splitext(file)[1] == '.npy']
    for k, file in tqdm(enumerate(files)):
        # load attention map
        att_map = np.load(os.path.join(data_path, file))
        # compute region scores
        scores[file] = region_scores(att_map, atlas_tsv, atlas_map, resize=True)
        if k == 2:
            break
    return scores

In [ ]:
scores = compute_scores('results/models/model_85/attribution_maps/GC/val/age', atlas_tsv, atlas_map)

In [ ]:
# sanity check: check if no attention map has NaNs
counter = 0
data_path = 'results/models/model_85/attribution_maps/GC/val/disease'
files = [file for file in os.listdir(data_path) if os.path.splitext(file)[1] == '.npy']
for k, file in tqdm(enumerate(files)):
    map_ = np.load(os.path.join(data_path, file))
    nans = np.isnan(map_).sum()
    if nans>0:
        counter+=1
        
print(counter/len(files))

In [ ]:
atlas_tsv['roi_name'][np.argsort(-importances)]

In [73]:
volumes = np.load("/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/model_85/predictions/test/age.npy")

In [56]:
def check_columns(index):
    l = []
    for k in range(4,len(index)):
        l.append(index[k].split('ROI')[1])
    return l

In [61]:
l1 = check_columns(raw_target_df.columns)
l2 = check_columns(raw_target_df2.columns)
for k in range(len(l1)):
    if l1[k] != l2[k]:
        print(k, l1[k], l2[k])

94 -Cerebelum_Crus1_L__intensity -Cerebelum_Crus1_L_intensity
102 -Cerebelum_6_L_intensity -Cerebelum_6_L_9_intensity


# random

In [ ]:
losses = pd.read_csv('train_losses_3D_2.csv')
test_losses = pd.read_csv('val_losses_3D_2.csv')

In [ ]:
losses

In [ ]:
plt.plot(test_losses[['disease', 'volumes', 'age', 'sex']])
plt.ylim([0,150])
plt.legend(['disease', 'volumes', 'age', 'sex'])

In [ ]:
plt.plot(losses[['disease', 'volumes', 'age', 'sex']])
plt.legend(['disease', 'volumes', 'age', 'sex'])

In [ ]:
import torch
from pprint import pprint
from torchmetrics import MetricCollection, Accuracy, Precision, Recall
target = torch.tensor([0, 1, 0, 1, 0, 1, 0, 1])
preds = torch.tensor([1, 1, 1, 0, 1, 1, 1, 1])
metrics = MetricCollection([Accuracy(),
                            Precision(num_classes=2, average='micro'),
                            Recall(num_classes=3, average='macro')])
metrics(preds, target)

# assess gradient norms

### Old method

In [ ]:
def get_gradient_norms(model): 
    path = 'results/models/model_{}/log.out'.format(model)
    
    file = None
    with open(path, 'r') as f:
        file = f.read()

    data = [ s[36:] for s in file.split('\n') if 'STDOUT' in s]
    data = [ s for k,s in enumerate(data[1:]) if data[k] == 'GRADIENT']
    data = np.array(data, dtype=np.float32)
    return data

def compute_metrics(data, name):
    dict_ = {'mean': data.mean(),
             'min': data.min(),
             'max': data.max(), 
             'std': data.std(), 
             'median': np.median(data)}
    return pd.DataFrame(dict_, index=[name])

def get_all_metrics():
    df = None
    for k in range(4):
        data = get_gradient_norms(9+k)
        if df is None:
            df = compute_metrics(data,BRANCH2TARGET['branch' + str(k+1)])
        else:
            df = df.combine_first(compute_metrics(data,BRANCH2TARGET['branch' + str(k+1)]))
    return df

def plot_training(model):
    fig, ax = plt.subplots(1,4,figsize=(16,4))
    for k, target in enumerate(TARGET2BRANCH):
        df_train = pd.read_csv('results/models/model_{}/train_losses.csv'.format(model))
        df_val = pd.read_csv('results/models/model_{}/val_losses.csv'.format(model))
        ax[k].plot(getattr(df_train, target), label='train')
        ax[k].plot(getattr(df_val, target), label='val')
        ax[k].set_title(target)
    plt.legend()
        

In [ ]:
df = get_all_metrics()
df

In [ ]:
weights = (1/df['mean']).to_numpy()
weights = weights/weights.sum()
weights

In [ ]:
plot_training(model=5)

In [ ]:
pd.read_csv('results/models/model_{}/val_losses.csv'.format(7))

### New method

In [ ]:
absolute_path = '/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models'
models = ['model_' + str(k) for k in range(21, 25)]
models = ['model_' + str(k) for k in [70, 73, 75, 76]]

In [ ]:
dfs = [pd.read_csv(os.path.join(absolute_path, model, 'gradient_norms.csv')) for model in models]

In [ ]:
norms = pd.concat(dfs, axis=1)
for key in TARGET2BRANCH.keys():
    norms.rename(columns={'gradient_norms_' + key: key}, inplace=True)

In [ ]:
stats = norms.describe()
stats

In [ ]:
stats.loc['mean']

In [ ]:
avg_norms = norms.rolling(window=153).mean()[0::153]
scaled_avg_norms = avg_norms/stats.loc['mean']

In [ ]:
avg_norms.plot()

In [ ]:
scaled_avg_norms.plot()

In [ ]:
scale_parameters = 1/stats.loc['mean']
scale_parameters = scale_parameters/scale_parameters.sum()*4
scale_parameters

nohup python training.py -bs 16 -lw 1.054807 1.607432 0.370006 0.967756 -d 0.2 &
nohup python training.py -e 100 --patience 100 -bs 16 -d 0.5 -lwp results/models/loss_weights.npy &

In [ ]:
# loss_weights = np.array([1.054807, 1.607432, 0.370006, 0.967756])
loss_weights = scale_parameters.to_numpy()

In [ ]:
np.save('results/models/loss_weights_t1l_without_bn.npy', loss_weights.astype(np.float32))

# Age investigation

In [ ]:
df = pd.concat([training_df, valid_df], ignore_index=True)
df2 = df_add_data.merge(df, on =['participant_id', 'session_id'], how='right')

In [ ]:
plt.title('Age Distribution')
plt.hist(training_df.age, label='training')
plt.hist(valid_df.age, label='validation')
plt.xlabel('age')
plt.legend()

# Results investigation

In [ ]:
absolute_path = '/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models'
models = [ model for model in os.listdir(absolute_path) if 'model' in model ]
dicts = []
for model in models:
    f = open(os.path.join(absolute_path, model, 'commandline.json'),)
    d = json.load(f)
    d.update({'model': int(model[6:])})
    dicts.append(d)
parameters = pd.DataFrame(dicts)
parameters.sort_values('model', inplace=True)
parameters.to_csv('results/models/summary.csv', index=False)
parameters

In [ ]:
parameters[-15:]

## performances

In [44]:
def get_performances(models):
    """
    Params:
        - models: list of ints (model numbers)
    """
    if type(models) != list:
        models = [models]
    
    best_performances = pd.DataFrame({'model': []})
    for model in models:
        try:
            performances = pd.read_csv('results/models/model_{}/test_metrics.csv'.format(model))
        except:
            saved_data = torch.load('results/models/model_{}/last_model.pt'.format(model))
            performances = pd.DataFrame(saved_data['val_metrics'])
        best_performances = best_performances.append(performances[performances.test == performances.test.min()])
    best_performances.reset_index(inplace=True)
    #best_performances.drop(columns=['index'], inplace=True)
    best_performances['model'] = models
    return best_performances

In [45]:
perf = get_performances([72, 77, 78, 79, 80, 81, 83, 85])
perf

,index,model,disease,volumes,age,sex,test,b1_Accuracy,b1_F1,b1_AUROC,b2_MeanSquaredError,b2_R2Score,b3_MeanSquaredError,b3_R2Score,b4_Accuracy,b4_F1,b4_AUROC
0,20,72,0.090923,0.221217,0.173681,0.052454,0.538274,0.888158,0.888889,0.946354,0.051961,0.341011,5.832608,0.326477,0.934211,0.933333,0.990303
1,17,77,0.100209,0.226368,0.176956,0.032029,0.459013,0.894737,0.888889,0.936458,0.052569,0.325496,5.887350,0.313775,0.953947,0.954839,0.991861
2,13,78,0.087246,0.252608,0.187973,0.041201,0.493580,0.868421,0.864865,0.930035,0.055642,0.247357,6.067847,0.271053,0.934211,0.933333,0.990823
3,13,79,0.099093,0.242389,0.203839,0.027808,0.478227,0.855263,0.842857,0.903472,0.054445,0.277635,6.318741,0.209525,0.947368,0.948718,0.993420
4,53,80,0.097928,0.237659,0.163061,0.033962,0.532610,0.855263,0.857143,0.930382,0.053951,0.291082,5.651477,0.367660,0.960526,0.960526,0.990649
5,6,81,0.103611,0.229576,0.184643,0.033362,0.551193,0.848684,0.832117,0.912326,0.053112,0.314953,6.013856,0.283967,0.947368,0.948052,0.989437
6,70,83,0.172959,0.230539,0.181573,0.006916,0.591986,0.526316,0.000000,0.500000,0.053107,0.312330,5.963646,0.295874,0.986842,0.986842,1.000000
7,51,85,0.116838,0.210712,0.177086,0.034424,0.461780,0.842105,0.844156,0.921528,0.050627,0.371804,5.889508,0.313272,0.953947,0.955414,0.993420


In [47]:
perf['R2Score'] = perf['b2_R2Score'] + perf['b3_R2Score']
perf['total_test'] = perf['disease'] + perf['volumes'] + perf['age'] + perf['sex']

In [48]:
perf

,index,model,disease,volumes,age,sex,test,b1_Accuracy,b1_F1,b1_AUROC,b2_MeanSquaredError,b2_R2Score,b3_MeanSquaredError,b3_R2Score,b4_Accuracy,b4_F1,b4_AUROC,R2Score,total_test
0,20,72,0.090923,0.221217,0.173681,0.052454,0.538274,0.888158,0.888889,0.946354,0.051961,0.341011,5.832608,0.326477,0.934211,0.933333,0.990303,0.667488,0.538274
1,17,77,0.100209,0.226368,0.176956,0.032029,0.459013,0.894737,0.888889,0.936458,0.052569,0.325496,5.887350,0.313775,0.953947,0.954839,0.991861,0.639271,0.535562
2,13,78,0.087246,0.252608,0.187973,0.041201,0.493580,0.868421,0.864865,0.930035,0.055642,0.247357,6.067847,0.271053,0.934211,0.933333,0.990823,0.518409,0.569029
3,13,79,0.099093,0.242389,0.203839,0.027808,0.478227,0.855263,0.842857,0.903472,0.054445,0.277635,6.318741,0.209525,0.947368,0.948718,0.993420,0.487160,0.573129
4,53,80,0.097928,0.237659,0.163061,0.033962,0.532610,0.855263,0.857143,0.930382,0.053951,0.291082,5.651477,0.367660,0.960526,0.960526,0.990649,0.658741,0.532610
5,6,81,0.103611,0.229576,0.184643,0.033362,0.551193,0.848684,0.832117,0.912326,0.053112,0.314953,6.013856,0.283967,0.947368,0.948052,0.989437,0.598920,0.551193
6,70,83,0.172959,0.230539,0.181573,0.006916,0.591986,0.526316,0.000000,0.500000,0.053107,0.312330,5.963646,0.295874,0.986842,0.986842,1.000000,0.608203,0.591986
7,51,85,0.116838,0.210712,0.177086,0.034424,0.461780,0.842105,0.844156,0.921528,0.050627,0.371804,5.889508,0.313272,0.953947,0.955414,0.993420,0.685075,0.539059


In [ ]:
model = 'model_72'
train_csv = pd.read_csv('results/models/{}/train_metrics.csv'.format(model))
val_csv = pd.read_csv('results/models/{}/test_metrics.csv'.format(model))

In [ ]:
val_csv['test'].plot()
train_csv['train'].plot()

In [ ]:
val = pd.read_csv('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/model_19/test_metrics.csv')
val

In [ ]:
val[val.test == val.test.min()]

In [ ]:
val[val.test == val.test.min()]

In [ ]:
val[val.test == val.test.min()]

## other

In [ ]:
#path = 'network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/model_32'
model = 'model_26'
training_df = pd.read_csv('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/{}/training_df.csv'.format(model))
valid_df = pd.read_csv('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/{}/valid_df.csv'.format(model))

train_transforms, all_transforms = get_transforms('image', minmaxnormalization=True, data_augmentation=None)
# fetch volumetric data
stds, df_add_data = fetch_add_data(training_df)

In [ ]:
caps_directory = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/'
# build MRI datasets
data_train = MRIDatasetImage(caps_directory, training_df, df_add_data=df_add_data, preprocessing='t1-volume',
                             all_transformations=all_transforms)  # train_transformations=all_transforms
data_valid = MRIDatasetImage(caps_directory, valid_df, df_add_data=df_add_data, preprocessing='t1-volume',
                             all_transformations=all_transforms)  # train_transformations=all_transforms,

In [ ]:
batch_size = 8

valid_loader = DataLoader(data_valid,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=8,
                          pin_memory=True)

train_loader = DataLoader(data_train,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=8,
                          pin_memory=True)

In [ ]:
model = 'model_26'
saved_data = torch.load(os.path.join('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/' + model, 'last_model.pt'))
# get sample
sample = data_train[0]
# build model
model = Net(sample, [8, 16, 32, 64, 128], 0.3).cuda()
# load pretrained weights on validation set
# saved_data = torch.load(os.path.join(args.model_path, 'test_best_model.pt'))
model.load_state_dict(saved_data['model_state_dict'])

In [ ]:
model.train()
model.training

In [ ]:
model.summary()

In [ ]:
resu = test(model,
     valid_loader,
     loss_weights= [1, 1, 1, 1], #np.array([1.054807, 1.607432, 0.370006, 0.967756]),
     to_cuda=True,
     rescaling=stds,
     compute_metrics=True,
     eval_mode=False)

In [ ]:
resu

In [ ]:
saved_data = torch.load(os.path.join('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/' + 'model_63', 'last_model.pt'))

In [ ]:
pd.DataFrame(saved_data['train_metrics'])

## investigating df_add_data

In [ ]:
# paths
data_path = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021.tsv'
summary_path = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021_summary.tsv'

# fetch indexes
df_summary = pd.read_csv(summary_path, sep='\t')
df_summary = df_summary[(df_summary.pipeline_name == pipeline_name) & (df_summary.atlas_id == atlas_id)]
first_column_name = df_summary.first_column_name.item()
last_column_name = df_summary.last_column_name.item()
print('First column name: ', first_column_name)
print('Last column name: ', last_column_name)
df_data = pd.read_csv(data_path, sep='\t', nrows=1)
first_column_index = df_data.columns.get_loc(first_column_name)
last_column_index = df_data.columns.get_loc(last_column_name)

# other data to fetch
col_names = ['participant_id', 'session_id', 'sex', 'age']
add_indexes = [df_data.columns.get_loc(col_name) for col_name in col_names]

# compute df_add_data
# add 1 to first_column_index to ignore background
used_columns = np.hstack([add_indexes, np.arange(first_column_index + 1, last_column_index + 1)]).flatten()
df_add_data = pd.read_csv(data_path, sep='\t', usecols=used_columns).dropna(axis=0, how='any')
print(df_add_data.head())

In [ ]:
# normalization using only statistics from training data
temp_df = pd.merge(training_data[['participant_id', 'session_id']],
                   df_add_data, on=['participant_id', 'session_id'], how='left')

In [ ]:
scalar_cols = [col for col in temp_df.columns if col not in ['participant_id', 'session_id', 'sex']]
# df_add_data[scalar_cols] contains only scalar columns with (patient, session) from training set
means, stds = temp_df[scalar_cols].mean(), temp_df[scalar_cols].std()
df_add_data[scalar_cols] = (df_add_data[scalar_cols] - means) / stds

In [ ]:
df_add_data[scalar_cols]

In [ ]:
pd.read_csv('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/model_39/train_metrics.csv')

In [ ]:
pd.read_csv('/network/lustre/dtlake01/aramis/users/sasha.collin/interpretability-dl-ndd/results/models/model_39/test_metrics.csv')

# Hyperparameters tuning

In [ ]:
path = 'results/models'
model_numbers = np.arange(47, 51)

In [ ]:
dfs = [pd.read_csv(os.path.join(path, 'model_' + str(k), 'test_metrics.csv')) for k in model_numbers]
dfs_train = [pd.read_csv(os.path.join(path, 'model_' + str(k), 'train_metrics.csv')) for k in model_numbers]

In [ ]:
for k in range(4):
    print(dfs_train[k]['train'].min(), dfs_train[k]['b2_R2Score'].max(), dfs_train[k]['b3_R2Score'].max())

In [ ]:
for k in range(4):
    print(dfs[k]['test'].min(), dfs[k]['b2_R2Score'].max(), dfs[k]['b3_R2Score'].max())

# AIBL

# RANDOM

I am using working with the file '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021.tsv' and using the 't1-volume' pipeline and the atlas 'AAL2'.
6267 samples (i.e. couples (participant_id, session_id)) do not have any volume value (i.e. NaN). 1 sample has no sex value and 1178 samples have no age values.
